In [1]:
import numpy as np
import mediapipe as mp
import cv2

In [ ]:
class Excersice:
    def __init__(self, s1, s2, s3):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
    def feedback(self, pose):
        if pose == self.s1:
            return 0
        elif pose == self.s2:
            return 1
        elif pose == self.s3:
            return 2
        else:
            return -1
        

In [2]:
def calculate_angle(a, b, c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


cap = cv2.VideoCapture('/Users/iyongbin/Repository/Lenfit/ML/Dataset/video/squat001.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False

        # detection
        results = pose.process(img)

        # draw landmarks
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # exception 
        try:
            body = dict()
            vis = list()
            isvis = 1
            landmarks = results.pose_landmarks.landmark
            
            for lndmrk in mp_pose.PoseLandmark:
                body[lndmrk] = [landmarks[lndmrk].x, landmarks[lndmrk].y, landmarks[lndmrk].z] # x, y, z, visibility:
                vis.append(landmarks[lndmrk].visibility)
            if vis.mean < 0.5:
                isvis = 0;
            print(body)
            print('-------------------')
            
            # print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )
        
        cv2.imshow('VideoFrame', img)
        if cv2.waitKey(1) == ord('q'):
            break



# 동영상 재생 종료
cap.release()
# 화면에 나타난 윈도우들을 종료
cv2.destroyAllWindows()


In [28]:
# # a list와 b list로 딕셔너리 만들기
# a = [str(i).strip().split('.')[1] for i in mp_pose.PoseLandmark]
# b = [i for i in range(len(mp_pose.PoseLandmark))]


# landmarks_dict = dict(zip(a, b))

# print(landmarks_dict)


{'NOSE': 0, 'LEFT_EYE_INNER': 1, 'LEFT_EYE': 2, 'LEFT_EYE_OUTER': 3, 'RIGHT_EYE_INNER': 4, 'RIGHT_EYE': 5, 'RIGHT_EYE_OUTER': 6, 'LEFT_EAR': 7, 'RIGHT_EAR': 8, 'MOUTH_LEFT': 9, 'MOUTH_RIGHT': 10, 'LEFT_SHOULDER': 11, 'RIGHT_SHOULDER': 12, 'LEFT_ELBOW': 13, 'RIGHT_ELBOW': 14, 'LEFT_WRIST': 15, 'RIGHT_WRIST': 16, 'LEFT_PINKY': 17, 'RIGHT_PINKY': 18, 'LEFT_INDEX': 19, 'RIGHT_INDEX': 20, 'LEFT_THUMB': 21, 'RIGHT_THUMB': 22, 'LEFT_HIP': 23, 'RIGHT_HIP': 24, 'LEFT_KNEE': 25, 'RIGHT_KNEE': 26, 'LEFT_ANKLE': 27, 'RIGHT_ANKLE': 28, 'LEFT_HEEL': 29, 'RIGHT_HEEL': 30, 'LEFT_FOOT_INDEX': 31, 'RIGHT_FOOT_INDEX': 32}


- squat 기준으로 피드백 발생기 만들기
- class로 만들어서 모든 운동 기준으로 바꾸기
- 접근이 용이하게 칼럼 선택하기

## 초기조건
- 카메라는 왼쪽을 기준으로
- 카메라는 화면에 수직으로 
  

In [48]:
# Excersice data


# left side
knee = [mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value]
hip = [mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value]
ankle = [mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value, mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
elbow = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST]
shoulder = [mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW]
wrist = [mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_INDEX]

squat_point = [knee, hip, ankle]

pushup_point = [elbow, shoulder, wrist, hip, knee]

left_armcurl_point = [elbow]

lunge_point = [knee, hip, ankle]

pullup_point = [elbow, shoulder, wrist, hip, knee]

# landmark에 연결된 부위 찾기
print(mp_pose.PoseLandmark.LEFT_HIP)

PoseLandmark.LEFT_HIP


In [50]:
## angle feedback function
def angle_feedback(angle_dict, state, limit_params, comments):
    fb_comments = list()
    if state == 'Squat':
        points = angle_dict.keys()
        for i, point in enumerate(points):
            comment = feedback(point, angle_dict[point], limit_params[i], comments[i])
            fb_comments.append(comment)
    elif state == 'Pushup':
        pass
    return fb_comments



def feedback(point, angle, limit, comment):
    print(point)
    if angle > limit:
        return comment
    else:
        return 'Good'

In [53]:
# parameter of excerise

# squart
squart_limit_params = [90, 90, 90]
squart_comments = ['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
state = 'Squat'

cap = cv2.VideoCapture('/Users/iyongbin/Repository/Lenfit/ML/Dataset/video/squat001.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False

        # detection
        results = pose.process(img)

        # draw landmarks
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # exception 
        # try:
        landmarks = results.pose_landmarks.landmark
        vis = list()
        
        # body dictionary 생성
        for i, lndmrk in enumerate(mp_pose.PoseLandmark):
            vis.append(landmarks[lndmrk].visibility)

        # check data visibility
        if np.mean(vis) < 0.5:
            continue
        angle = dict()

        # calculate angle
        if state == 'Squat':
            for points in squat_point:
                xy_data = [[landmarks[point].x, landmarks[point].y] for point in points]
                angle[points[1]] = calculate_angle(xy_data[0], xy_data[1], xy_data[1]+[0, 1]) # vertical angel
                fb = angle_feedback(angle, state, squart_limit_params, squart_comments)
        elif state == 'Pushup':
            pass
        elif state == 'Lunge':
            pass
        elif state == 'Pullup':
            pass
        elif state == 'ArmCurl':
            pass
        # except:
        #     print('error')
        print(fb)

        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )
        
        cv2.imshow('VideoFrame', img)
        if cv2.waitKey(1) == ord('q'):
            break



# 동영상 재생 종료
cap.release()
# 화면에 나타난 윈도우들을 종료
cv2.destroyAllWindows()


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
